# Survey DOI Extraction

A survey was performed and all EC PIs were invited to update the spreadsheet provided in Google Docs so that their projects were properly represented.

In this notebook, we extract those mappings to make sure that those publications not represented in the NSF data properly accounted for.

In [1]:
import openpyxl as pyxls # https://openpyxl.readthedocs.io/en/stable/usage.html

In [2]:
# wb = pyxls.load_workbook(filename = '050922_ec_projects_worksheet_distrubute.xlsx')
wb = pyxls.load_workbook(filename = '../inputs/052522_ec_projects_worksheet_distrubute.xlsx')

Building a project map allows us to match ID to names -- the names in the spreadsheet were different than those give by the NSF, so the origin file is use to walk from ID to EC program name.  Recall, the NSF database program names were NOT always an exact match for those in the EC database, so the NSF Award ID is the only key we have to work with.

In [3]:
prj_map = {}
for sheet in wb.sheetnames:
    i = 2
    while (True):
        doi, citation = wb[sheet]['B'+str(i)].value, wb[sheet]['D'+str(i)].value
        
        if doi is not None or citation is not None:  
            if doi:
                value = doi.strip()
            else:
                value = citation.strip()
                
            if not prj_map.get(sheet):
                prj_map[sheet] = [value]
            else:
                prj_map[sheet].append(value)
            i+=1
        else:
            print(".", end="")
            break
            

................................................................................................

In [4]:
import pandas as pd
eco_names = \
    pd.read_csv("../outputs/eco_funded_project_list.tsv", sep='\t')

In [5]:
eco_names['crosswalk'] = eco_names.program \
    .str.replace(":", "") \
    .str[:15] \
    .str.lower()

In [6]:
eco_names.head()

,nsfid,shortname,program,crosswalk
0,1324760,RCN,Building a Sediment Experimentalist Network (SEN),building a sedi
1,1340233,Office,EarthCube Test Enterprise Governance: An Agile...,earthcube test
2,1340265,BB,EC3: Earth-Centered Communication for Cyberinf...,ec3 earth-cente
3,1340301,RCN,C4P: Collaboration and Cyberinfrastructure for...,c4p collaborati
4,1343661,CD,A Scalable Community Driven Architecture,a scalable comm


Using the NSF ID as the key, write the new files so they can ultimate be merged.  The unmatched file can be explored manually.

In [7]:
import re

with open("../outputs/ec_survey_project_doi_mapping.tsv", "w", encoding='utf=8') as fo1, \
     open("../outputs/ec_survey_project_unmatched.tsv", "w", encoding='utf=8') as fo2:
    fo1.write(f"nsfid\tdoi\n")
    fo2.write(f"nsfid\tunmatched_citation\n")

    for k, v in prj_map.items():
        k_ = re.sub(r"^\d+-", "", k)
        nsfid = eco_names.query(f'crosswalk == "{k_[:15].lower()}"')['nsfid'].values[0]

        if nsfid:
            for i in v:
                if i[:3].strip() == "10.":
                    fo1.write(f"{nsfid}\t{i.strip().lower()}\n")
                else:
                    fo2.write(f"{nsfid}\t{i}\n")
        else:
            print("[warn]: nsfid not found")